# Konda - Quick Start Guide

## 🐍 Conda Installer/Wrapper for Google Colab 🚀

This notebook provides a quick start guide to using Konda in Google Colab.

## Installation

In [ ]:
!pip install konda

## Setup Miniconda

In [ ]:
import konda
konda.install()

## Basic Usage

Konda is a simple wrapper around conda. Just replace `conda` with `konda` in any conda command.

In [ ]:
# Create an environment
!konda create -n tf_env python=3.8 -y

# Activate an environment
!konda activate tf_env

# Run commands in the activated environment
!konda run "pip install tensorflow==2.4.0"
!konda run "python -c 'import tensorflow as tf; print(tf.__version__)'"

## Complete Example: Setting up a TensorFlow Environment

In [ ]:
%%writefile tf_mnist.py
# Training a neural network on MNIST with Keras: https://www.tensorflow.org/datasets/keras_example
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

In [ ]:
!konda create -n tf_env python=3.8 -y
!konda activate tf_env
!konda run "conda install anaconda::tensorflow-gpu=2.4.1 -y && pip install tensorflow_datasets"
!konda run "python tf_mnist.py"

## Cleanup

In [ ]:
# Deactivate
!konda deactivate

# Remove Miniconda installation when done
!konda uninstall

That's it! Konda makes conda easy to use in Google Colab.